# 7章 戦略
担当: 須藤

## 冒頭

本書のここまでの部分では、 pytest のメカニズムについて説明してきました。 pytest の メカニズムは、ソフトウェアテストの「どのようにテストを書くか」 にあたる部分（テスト関数の記述・フィクスチャの使用・パラメータ化されたテストの実装）

本章では、pytest について学んできたことをすべて使って、Cards プロジェクトのテスト戦略を作成する。テスト戦略の作成は、ソフトウェアテストの「どのようなテストを書くか」にあたる部分である。

テスト戦略はまずテストスイート（テストの目的や条件が似ている複数のテストケースを一括りにしたもの）を以下の手順で決める:
1. テストスイートの目標を定義する
2. Cards プロジェクトのソフトウェアアーキテクチャがテスト戦略にどのような影響を与えるのか、テストのニーズからどのような影響を受けるのかを探る
3. テストする機能を選択し、優先順位を決め、必要なテストケースをリストアップする

本章の内容はソフトウェアテスト戦略全体を完全にカバーするものではないが、1つのプロジェクトで考えられるテスト戦略を調べておけば、 自分のプロジェクトにとって最適なテスト戦略を判断できるようになるだろう


## 7.1 テストの範囲を決める

テストの目標と要件はプロジェクトごとに異なる。心拍数モニタリングシステム等、あらゆるレベルで徹底的なテストが求められるものがある一方、アニメーション GIF を作成するためのツール等もある。ほとんどのソフトウェアはその中間にある。

ユーザーから見える機能の振る舞いはほぼ例外なくテストしたほうが良いだろう。しかし、どこまでテストする必要があるかを判断するときには、検討しなければならない問が他にもいろいろある。

- セキュリティ
  - セキュリティ対策は必要か。機密情報を保存する場合は特に重要
- パフォーマンス
  - インタラクションは高速にすべきか。 速度的にはどれくらいか
- 負荷
  - 大勢のユーザーからの大量のリクエストに対処できる必要があると見込む場合、対処できるようにする
- 入力バリデーション
  - ユーザー入力を受け取るシステムはどのようなものであれ、そのデータを使う前に検証を行うべきである。



プロジェクトの規模が大きくなれば当然必要なものであるが、（個人または小さなチームで使うことを想定している）Cards プロジェクトでも、上記の問題はどれも当てはまる。では、最初のテストスイートでは、テストをどれくらい行うべきだろうか。最初は次のようにするのが妥当だろう。
- ユーザーから見える機能の振る舞いをテストする。
- 現在の設計のセキュリティテスト、パフォーマンステスト、負荷テストは後回しにする。現在の設計では、データベースはユーザーのホームディレクトリに格納される。このデータベースを複数のユーザーによって共有される場所に移動する。としたら、上記の問題の重要性は間違いなく高まるだろう。
- Cards がシングルユーザーアプリケーションである間は、入力の検証もそれほど重要ではない。しかし、アプリケーションの使用時にスタックトレースが表示されるのは避けたいため、少なくとも CLI レベルで不正な入力をテストすべきである。

どんなプロジェクトでも必要となるのは機能テストである。しかし、機能テスト1つをとっても、どの機能をどの優先順位でテストするのかを決めないといけなく、機能ごとにテストケースを決める必要もある。

例として、Cards プロジェクトでこの作業を最初から最後まで行うことにしよう。
まず機能の優先順位を決定し、テストケースを生成することから始める。しかし、プロジェクトのソフトウェアアーキテクチャがテスト戦略の選択にどのような影響を与えるのかが気になる。 
作業に取りかかる前に、この点を調べておこう


### コラム: 安眠のためのテスト

夜ぐっすり眠れるくらい十分なテストという発想は、真夜中にソフトウェアが動かなくなったら開発者が呼び付けられるソフトウェアシステムがきっかけで生まれたのかもしれません。

今では、その意味するところは、ソフトウェアが十分にテストされていることがわかっているので枕を高くして眠れるというところまで拡大されています。

かなり大雑把な考え方ですが、 以降の節で、 どの機能をテストすべきか、どのテストケースが必要かを評価するときに、この発想が助けになるでしょう。

## 7.2 ソフトウェアアーキテクチャについて考える

アプリケーションがどのような構成になっているか --- つまり、そのアプリケーションのソフトウェアアーキテクチャは、テスト戦略を決めるときに重要なポイントとなる。
以下はどれもソフトウェアアーキテクチャの一部である: 
- プロジェクトのソフトウェアがどのような構成になっているか
- どのようなAPIが利用できるか
- どのようなインターフェイスがあるか
- コードの複雑な部分がどこにあるか
- どのようにモジュール化されているか
  
テスト絡みでは、システムのどれだけの部分をテストしなければならないか、そしてエントリポイントがどこにあるかを知る必要がある。

単純な例として、あるモジュールに存在するコードをテストしているとしよう。このモジュールはコマンドラインで使うためのもので、出力を書き出すもの以外にインタラクティブコンポーネントはなく、API もない。また、このモジュールは Python で書かれていない。このような場合、ブラックボックスとしてテストするしかない。つまり、テストコードから様々なパラメータやっ状態を使ってモジュールを呼び出し、その出力を調べることとなる。

そのコードが Python で書かれていて、インポート可能で、モジュール内の関数を呼び出してさまざまな部分をテストできる場合、私たちには選択肢がいくつかある。このモジュールもやはりブラックボックスとしてテストできるが、モジュール内の関数を別々にテストしたければそうすることもできる。

もっと大規模なシステムでも同じように考えることができる。テスト対象のソフトウェアが Python パッケージとして設計されていて、大量のサブモジュールで構成されているとしよう。この場合もやはり CLI レベルでテストできるが、もっと細かいレベルでもテストできる。たとえば、モジュールごとに、あるいはモジュール内の関数ごとにテストできる。連動するサブシステムとして設計されたさらに大規模なシステムでは、それぞれのサブシステムが複数のパッケージやモジュールを使っているかもしれない。


- どのレベルでテストすべきか。 トップレベルのユーザーインターフェイスか。それとももっと低いレベルか。 サブシステムレベルでテストすべきか。すべてのレベルでテストすべきか。
依存しています。
ルでさまざまな書式 式を整えるため べてのロジックが データベースの
ケージを使って
てあります。
すべ
- さまざまなレベルでのテストはどれくらい簡単か。最も難しいのはたいてい、UI のテストだが、顧客向けの機能と結び付けるのが容易な部分でもある。個々の関数のテストを実装するのは簡単かもしれないが、顧客の要件と結び付けるのはそれほど簡単ではない。
- さまざまなレベルとそれぞれのテストを担当するのは誰か。あなたが担当しているのがサブシステムである場合、あなたがテストしなければならないのはそのサブシステムだけだろうか。システムのテストは他の誰かが行うのだろうか。そうであれば選択は簡単で、あなたは自分のサブシステムをテストすればよいとはいえ、少なくともシステムレベルで何がテストされているのかを知った上で作業に取りかかるのがよいだろう。

話を少し単純にするために、あなたとあなたのチームが何もかも担当していて、ソフトウェアが階層構造になっているとしよう。一番上にある UI 層はロジックがほとんどない超薄い層であり、API 層やシステム内の他の部分を呼び出すようになっている。コードの残りの部分は巨大な 1つのファイルかもしれないし、サブシステムやモジュールとしてうまく設計されているかもしれない。

となると、システムテストは実質的に API 層に対するテストになる。UI 層については、API を正しく呼び出すことを確認する最低限のテストを行うことになるだろう。つまり、システムテストとして UI 層で高レベルのテストを行った後、API層を集中的に テストすればよいはずである。

この単純なシステムはCards プロジェクトのシステムそのものである。Cards プロジェクトは次の3つの層で実装されている。
1. CLI層 (cli.py)
2. API層 (api.py)
3. DB層 (db.py)

CLI層は clipy で実装されており、 Typer と Rich の2つのサードパーティパッケージに依存している。
- Typer: CLI の構築に使っているツール
- Rich はターミナルでさまざまな書式設定を行うことができるライブラリ。ここでは単にテーブルの書式を整えるために使っている。

CLI は意図的にできるだけ薄い層にしており、ほぼすべてのロジックが API 層に配置されている。

データベースの操作は db.py で実装されており、TinyDB というサードパーティバッケージを使っている。TinyDB が実際のデータベースである。この層もできるだけ薄くしてある。

cli.py と db.py の両方をできるだけ薄くしているのには、次の2つの理由があります。

- APIを使ったテストにより、システムとロジックのほとんどがテストされる。
- サードパーティパッケージへの依存が1つのファイルに隔離される。

サードパーティパッケージを隔離することにはさまざまな利点がある。それらの依存パッケージのインターフェイスが変更されたために何らかの変更が必要になったとしても変更しなければならないファイルは1つだけである。依存パッケージを何か別のものに置き換える場合も、変更しなければならないファイルはやはり1つだけである。たとえば、別のデータベースバックエンドを試してみたい場合は、エントリポイントとして db.py を使うテストスイートを作成し、データベースを変更し、db.py でデータベースアダプタを書き換えるだけでよいはずです。

Cards プロジェクトの cli.py を薄く保っている主な理由は、そのようにするとほとんどのテストのターゲットを API にできるからです。db.py を薄く保っている主な理由は、そのようにするとデータベースの期待値のテストを分離できるようになるからである。

- CLI のロジックはほんのわずかなので、ほぼすべての部分のテストに API を使うことができる。
- CLI のテストでは、 CLI が API の正しいエントリポイントを呼び出すことを確認できれば、それで十分なはずである。
- データベースの操作は db.py に分離されているため、サブシステムのテストが必要であると感じた場合はそのテストを DB 層に追加できる。

API を使ってテストを行うとしても、テストを実装することに夢中になるのではなく、エンドユーザーから見える振る舞いをテストすることに集中したい。したがって、Cards プロジェクトにとってうまくいくテスト戦略は次のようなものになる:
- ユーザーが利用できる (CLI でユーザーに表示される) 機能をテストする
- それらの機能のテストには (CLIではなく) API を使う
- CLI については、 CLI が API に正しく接続していることを確認するのに十分なテストを行う。
出発点としてはよさそうです。分離した状態でのデータベースのテストはひとまず後回しにできる。次は、何をテストするのかを決めるために、ユーザーから見える機能を調べてみよう。


## 7.3 テストする機能を調べる

テストケースを作成するには、まずどの機能をテストするのかを調べる必要がある。テストする機能の数が多い場合は、テストの開発に優先順位を付ける必要がある。少なくともだいたいの順序がわかっていると助けになる。筆者はたいてい次の5つの基準に基づいてテストする機能の優先順位を決めている:

- Recent
    新しい機能、コードの新しい部分、および修復、リファクタリング、変更を最近行った機能
- Core
    製品の USP (Unique Selling Proposition)。つまり、それらが動作していないと製品の有効性が損なわれるような機能
- Risk
    アプリケーションにおいてリスクの大きい部分。たとえば、顧客にとって重要だ が開発チームがあまり使わない部分や、あまり信頼できないサードパーティのコードを使っている部分
- Problematic
    誤動作が多い機能、または不具合がよく報告される機能
- Expertise
    限られた人だけが理解している機能またはアルゴリズム


Cards アプリケーションにおけるエンドユーザーから見える機能を調べてみよう。

```bash
$ cards --help
Usage: cards [OPTIONS] COMMAND [ARGS] ...
    Cards is a small command line task tracking application.

Options:
    --help Show this message and exit.

Commands:
    add Add a card to db.
    config List the path to the Cards db.
    count Return number of cards in db.
    delete Remove card in db with given id. Set a card state to 'done'.
    list List cards in db.
    start Set a card state to 'in prog'.
    update Modify a card in db with given id with new info. Return version of cards application
    version Return version of cards application
```

ここでは、 Cards プロジェクトをテストが必要なレガシーシステムとして扱っているため、5つの基準の中でも特に次の2つが役立つ。

- Core

    add、count delete finish、 list start update はどれも中核的な機能 に思えます。 config と version はそれほど重要ではないように思えます。

- Risk
  
    サードパーティパッケージは、CLI で使っている Typer とデータベースで使っている TinyDB の2つである。これらのパッケージを使っている部分を集中的にテストするのが賢明に思える。Typer を使っている部分のテストは CLI のテストで 行う。TinyDB を使っている部分のテストは、実際には他のすべてのテストで行うことになる。TinyDB の操作は db.py に分離されているため、必要で あれば DB層に焦点を合わせたテストを作成できます。

この場合は機能の数が少ないので、 Cards プロジェクトのすべての部分を実際にテストすることになる。ただし、このように機能をざっと分析するだけでも戦略を立てるのに役立つ。

- 中核的な機能を徹底
- それ以外の機能についても少なくとも1つのテストケースでテストする
- CLI を分離した状態でテストする
 
では、この戦略に基づいてテストケースを作成してみよう


## 7.4 テストケースを作成する

テスト戦略の目標と範囲が決まったところで、 テストケースの作成も同じように行えば簡単である。最初のテストケースの作成には、次の基準が役立つ: 

1. 最初に、"自明ではない"「ハッピーパス」 テストケースを作成する。
   - ハッピーパス: 例外やエラーの状態のないデフォルトのシナリオのこと。すべてが期待どおりに進んだ場合に実行される一連のアクティビティで構成される([参考](https://experienceleague.adobe.com/ja/docs/experience-manager-65/content/managing/managing-best-practices-checklist/best-practices-glossary))。
2. 次に、以下の要素に対するテストケースを調べる:
   - 興味深い入力
   - 興味深い開始状態
   - 興味深い終了状態
   - エラー状態として考えられるものすべて
  
テストケースが上記の基準を2つ以上満たしていれば問題ない。Cards プロジェクトの機能がどのようなものか理解するために、実際にいくつか試してみよう。


### 例1: count コマンドについて

count コマンドはコマンドライン上で `cards count` のように実行でき、DB の数を出力するコマンドである

1. count コマンドのハッピーパステストケースとして「データベースが空の場合、count は 0 を返す」 が考えられる。ただしこれは"自明な"テストケースと考えられるものでもあり、また、count コマンドが 0 を返すようにハードコーディングされていたらどうなるか、といった問題もある。そこで、ハッピーパステストケースのうち自明ではないテストケースとして次のものを考える: 

   - データベースに3枚のカードが含まれている場合、 count は 3 を返す

2. 
- "興味深い入力": count コマンドにはパラメータがないため、該当するものは無い。
- "興味深い開始状態": 次の3つの状態が考えられる。
  - データベースが空である
  - データベースにカードが1枚含まれている
  - データベースにカードが何枚か含まれている
- "興味深い終了状態": count コマンドはデータベースを書き換えないため、該当なし
- "エラー状態": 該当なし
  
以上から、count コマンドのテストケースは次の3つになる: 
- データベースが空のときの count
- データベースにカードが1枚含まれているときの count
- データベースにカードが何枚か含まれているときの count

最後のテストはハッピーパステストケースに該当するため、この3つで十分だろう。実際このように、2.の基準に基づいて生成されたテストケースの1つによって1.で考えたハッピーパスが満たされることがよくある。

1.で自明ではないハッピーパステストケースについてわざわざ考えるのはなぜだろうか。その理由は2つある。
1. 急いでいるときは、テストしている機能ごとに、自明ではないハッピーパステストケースを1つだけ作成するという手があることである。テストケースとしては完全では無いが、最低限の作業でシステムの大部分をテストするのにかなり効果的である ※ 筆者自身、このようにしてハッピーパステストケースを作成し、あとからテストケースを追加するという方法をとったことが何度かあった。

2. ハッピーパスから始めるもう1つの理由は、他の基準について考えるのがかなり楽になることである。うまくいかない可能性があるものから始めると、うまくいくケースのテストを忘れてしまうことがある。


### 例2: add コマンドについて

```bash
$ cards add --help
Usage: cards add [OPTIONS] SUMMARY ...
    Add a card to db.

Arguments:
    SUMMARY ... [required]
    
Options:
    -o, --owner TEXT
    --help      Show this message and exit.
```

自明ではないハッピーパステストケースの一つに、「空ではないデータベースにカードを追加する」が考えられる。

add は上の --help によれば、サマリー (summary) は必ず指定しなければなりませんが、所有者 (owner) は指定しなくてもよいことになっている（実際の格納のされ方はp15 等参照）。したがって、サマリーだけを指定するケースと、サマリーと所有者の両方を指定するケースをテストすべきである。サマリーを指定しない場合はどうなるだろうか。その場合はエラー状態に分類される。所有者が空のテキストである場合も同様である 新たに追加するカードのサマリーと所有者が既存のカードのものと同じである場合はどうなるだろうか。そのカードを追加すべきか、あるいはエラー状態として拒否すべきか。この質問は開発時にテストを書くことの価値を示している。あるいは少なくとも、振る舞いや API が変わると既存のユーザーを混乱させるため簡単には変更できないという状態になる前にテストを書くべきである。では、このカードは追加すべきか、拒否すべきか。Cards アプリケーションはカードの重複を許可するが、どちらの答えも妥当であるが、いずれにせよそのテストを行うべきである。

add コマンドのテストケースは次のようになる。
- サマリーを指定したカードを空のデータベースに add する
- サマリーを指定したカードを空ではないデータベースに add する
- サマリーと所有者を両方とも指定したカードを add する
- サマリーを指定せずにカードを add する
- 重複するカードを add する

※ 以下、書籍では delete 以降のテストケースについても書かれているが割愛

これらのテストケースがテストの詳細な説明ではないことに注目(注意)してほしい。テストケースを実装していて、本当に正しい振る舞いとは何だろうという疑問が湧いてくることがあるが、それはよいことである。そういった疑問はコミュニケーション、設計の明確化、API の完全性を促進するきっかけになることがよくある。また、ドキュメントの不備を発見するのに役立つこともある。

最初のテストケースのリストは完全ではない。テストを書いていると決まって新たなテストケースを思い付くものである。チームで作業を行っている場合は、チームの意見を聞くのに絶好のタイミングである。この段階のテストケースは正式なものではないため、コードの細かい部分にこだわることなく振る舞いについて話し合うことができる。

テストを完成させるために必要な情報がまだ足りないこともある。たとえば、例外が想定されている場合、その例外は具体的にどのようなものになるだろうか。 テスト対象のコードの API がまだ完成していない場合は特にだが、不足している情報があっても問題は無い。この段階でテストケースのリストについて特定分野のエキスパートと話し合っておけば、テストを書いていて細かい部分に疑問が生じたとしても、彼らがすぐに答えてくれるはずである。

この「テストする機能を調べて最初のテストケースをリストアップする」 という計画作業が完了したら、さっそくテストの作成に進みたくなるだろう。しかし、いったん立ち止まって、これまでの取り組みの内容を書き出しておくことを勧める。


## 7.5 テスト戦略を書き留める

本章では、テストのほとんどを、API を使って行うと決めた。CLI については、CLI が API を正しく呼び出すことを確認するのに十分なテストを行う。データベースのテストはひとまず先送りにします。その続きは、新しいデータベースパッケージに移行するのに役立つテストが必要になったときに再開できる。

このテスト戦略はかなりざっくりまとめたものであるが、この程度の戦略でさえ、ひとたびテストが始まると細かい部分をすぐに忘れてしまう。このため、筆者はあとから確認できるようにテスト戦略を書き留めておくことにしている。チームで作業を行っている場合は、たとえ2人だけのチームであっても、テスト戦略を書き留めておくことが特に重要となる。

Cards プロジェクトの現時点のテスト戦略は次のとおり: 
- エンドユーザーインターフェイス (CLI) を使ってアクセスできる振る舞いと機能をテストする。
- これらの機能のテストにはできるだけ API を使う。
- CLI については、すべての機能でAPIが正しく呼び出されることを検証するのに十分なテストを行う。
- 7つの中核的な機能 (add, count, delete, finish、 list, start, update) を完全にテストする。
- config と version の大まかなテストを追加する。
- TinyDB を使っている部分のテストは db.py に対するサブシステムテストで行う。

なお、このリストには含まれていないが、ドキュメントや社内 wiki などを使ってテスト戦略をチームで共有する場合は、最初のテストケースのリストを必ず盛り込むべきである。

おそらく、テストの進行に伴って、この最初のテスト戦略を拡張することになるだろう。変更が必要だと感じたらいつでもチームで話し合うことをお勧めする。

テストする機能、最初のテストケースのリスト、テスト戦略を書き出すための時間は先行投資であり、次のステップであるテストの実装に突入すれば、すぐに元が取れる。

Note: ダウンロードサンプルの code/ch7 にテストケースコードが含まれているので目を通しておくと良い